In [1]:
import pandas as pd
from sklearn.preprocessing import minmax_scale
import numpy as np
from scipy.stats import multivariate_normal
import sklearn as sk
from sklearn import covariance
from sklearn.metrics import confusion_matrix, classification_report
import math

def distance(x,y):
    n = len(x)
    d = 0
    for i in range(n):
        d = d+(x[i]-y[i])**2
    return d**0.5


def Knnprob(x_0,cl,k):
    n = len(x_0)  #Dimension
    num = len(cl) #number pf samples
    dist = np.zeros(num)
    dist2 = np.zeros(num)
    for i in range(num):
        dist[i] = distance(x_0,cl[i])
        dist2[i] = distance(x_0,cl[i])
        
    dist.sort()
    X_F = cl[list(dist2).index(dist[k])]
    H = np.array([abs(x_0[i]-X_F[i]) for i in range(n)])
    denum = num*H.max()**n
    
    return k/denum
    
trainData = pd.read_csv('pendigitstrain.csv')
testData = pd.read_csv('pendigitstest.csv')

TrainLabel = trainData['17']
TestLabel = testData['17']

TrainFeat = trainData.drop(['17'],axis = 1)
TestFeat = testData.drop(['17'],axis = 1)

TrainFeatScaled = minmax_scale(TrainFeat, feature_range = (0,1))
TestFeatScaled = minmax_scale(TestFeat, feature_range = (0,1))


def decision(x, X, Y):
    probability_List = []
    num_classes = len(np.unique(Y))
    shrinkage = 0.15
    N_T = len(X)
    k = 10
    for i in range(num_classes):
        X_train = X[Y==i]
        probability = Knnprob(x,X_train,k)*(len(X_train)/N_T)
        probability_List.append(probability)
    return probability_List.index(max(probability_List))


TestDecision = np.zeros((len(TestLabel),1))
CCR = 0
for i in range(len(TestDecision)):
    x = TestFeatScaled[i][:]
    TestDecision[i][0]=decision(x, TrainFeatScaled, TrainLabel)
    if TestDecision[i][0] == TestLabel[i]:
        CCR += 1



print(CCR/len(TestLabel))

print(confusion_matrix(TestDecision, TestLabel, [0, 1,2,3,4,5,6,7,8,9]))
print(classification_report(TestDecision, TestLabel))


0.938822184105203
[[354   0   0   0   0   0   0   0   5   0]
 [  2 298   5   6   0   0   0  14   1   8]
 [  1  47 358   0   4   0   0   2   0   0]
 [  0   1   0 327   1  10   0   3   0  20]
 [  0   1   0   0 341   0   0   2   0  11]
 [  0   6   0   1  14 320   1   0   6   3]
 [  1   2   0   0   1   0 333   0   0   0]
 [  0   6   1   0   1   0   1 340   1   4]
 [  5   0   0   0   0   0   1   3 323   0]
 [  0   3   0   2   2   5   0   0   0 290]]
              precision    recall  f1-score   support

         0.0       0.98      0.99      0.98       359
         1.0       0.82      0.89      0.85       334
         2.0       0.98      0.87      0.92       412
         3.0       0.97      0.90      0.94       362
         4.0       0.94      0.96      0.95       355
         5.0       0.96      0.91      0.93       351
         6.0       0.99      0.99      0.99       337
         7.0       0.93      0.96      0.95       354
         8.0       0.96      0.97      0.97       332
         9

C:\Program Files\Anaconda3\lib\site-packages\sklearn\utils\validation.py:71: FutureWarning: Pass labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
